# Cosi-Corr

## Setup

In [1]:
%matplotlib inline
import os
import glob
from pathlib import Path, PureWindowsPath
import subprocess
import itertools

from osgeo import gdal
from datetime import datetime
import shutil
import matplotlib.pyplot as plt
import pandas as pd
from osgeo import gdal
from collections import namedtuple

In [2]:
pwd

'/mnt/i/sentinel2/pixeloffset_mintpy'

In [3]:
cd /mnt/i/planet/andrew

/mnt/i/planet/andrew


In [4]:
Image = namedtuple('Image','date file')

In [5]:
types = {'planet':{'files':'*panchromatic_AnalyticMS_SR*tif','date_loc':slice(0,8)},
        'sentinel2':{'files':'s2*tif','date_loc':slice(-12,-4)}}

meta = types['planet']
pattern = meta['files']
date_loc = meta['date_loc']

In [6]:
names = [Image(os.path.basename(x)[date_loc],x) for x in glob.glob(pattern)]

merged = [x for x in names if 'merged' in x.file]
for m in merged:
    names = [x for x in names if m.date != x.date]

names += merged
names

[Image(date='20200903', file='20200903_panchromatic_AnalyticMS_SR_clip.tif'),
 Image(date='20210910', file='20210910_panchromatic_AnalyticMS_SR_clip.tif'),
 Image(date='20170911', file='20170911_panchromatic_AnalyticMS_SR_clip.tif'),
 Image(date='20180909', file='20180909_panchromatic_AnalyticMS_SR_clip.tif'),
 Image(date='20190918', file='20190918_panchromatic_AnalyticMS_SR_clip.tif')]

## Planet

In [61]:
dates = [x.date for x in names]

for d in set(dates):
    if dates.count(d) > 1:
        print(f'merging {d}...')
        files = [x.file for x in names if x.date == d]
        out_name = '_'.join(files[0].split('_')[:1]+['merged']+files[0].split('_')[-3:])
        !gdal_merge.py {' '.join(files)} -o {out_name}

merging 20180909...
0...10...20...30...40...50...60...70...80...90...100 - done.
merging 20190918...
0...10...20...30...40...50...60...70...80...90...100 - done.
merging 20170911...
0...10...20...30...40...50...60...70...80...90...100 - done.


## Run Cosi-Corr

In [7]:
geo = []

for n in names:
    ds = gdal.Open(n.file, gdal.GA_ReadOnly)
    geo.append([ds.RasterXSize,ds.RasterYSize,ds.GetGeoTransform()])
    ds = None


if max([len(list(set(x))) for x in zip(*geo)]) == 1:
    print('All geotransforms the same!')
else: 
    print('All geotransforms the NOT same!')
    print(geo)

All geotransforms the same!


In [8]:
files = [sorted(x) for x in itertools.combinations([y.file for y in names],2)]
# files = [sorted(x) for x in itertools.combinations(names.values(),2) if x[0][-12:-8] != x[1][-12:-8]]

len(files)

10

In [9]:
# pairs = pd.read_csv('pairs.csv')
# files = list(zip(pairs.image1,pairs.image2))
# len(files)

In [11]:
# if using bounds:
#for bounds, sample=x, line=y
#xstart, xend, ystart, yend
poroa_upper = '9873 10773 9570 10773'
no_bounds = ''

bounds = no_bounds
band = 1
data_dir = PureWindowsPath('I:/planet/andrew')
out_dir = data_dir / 'cosicorr_sobel'
window = 64
step = 16

# data_dir = PureWindowsPath(data_dir)
# out_dir = PureWindowsPath(out_dir)

dic = {'primary':[f'{str(data_dir / x[0])} {band} {bounds}' for x in files],
       'secondary':[f'{str(data_dir / x[1])} {band} {bounds}' for x in files],
       'out_name':[str(out_dir / f'cosicorr_{x[0][date_loc]}T000000_{x[1][date_loc]}T000000.dat') for x in files]}

for key in dic:
    with open(f'{key}.txt', 'w') as f:
        for line in dic[key]:
            f.write(line)
            f.write('\n')

# frequency, winMaxX, winMaxY, winMinX, winMinY, stepX, stepY, nbRobustIteration, MaskThreshold, resampling, grid
options = [f'frequency {window} {window} {window} {window} {step} {step} 5 0.9 0 0'] * len(files)
with open('options.txt', 'w') as f:
    for line in options:
        f.write(line)
        f.write('\n')

## Prep for MintPy

In [12]:
cd /mnt/i/planet/andrew/cosicorr_sobel

/mnt/i/planet/andrew/cosicorr_sobel


In [13]:
# create separate tif files for each band
files = glob.glob('*.dat')
files.sort()
files

['cosicorr_20170911T000000_20180909T000000.dat',
 'cosicorr_20170911T000000_20190918T000000.dat',
 'cosicorr_20170911T000000_20200903T000000.dat',
 'cosicorr_20170911T000000_20210910T000000.dat',
 'cosicorr_20180909T000000_20190918T000000.dat',
 'cosicorr_20180909T000000_20200903T000000.dat',
 'cosicorr_20180909T000000_20210910T000000.dat',
 'cosicorr_20190918T000000_20200903T000000.dat',
 'cosicorr_20190918T000000_20210910T000000.dat',
 'cosicorr_20200903T000000_20210910T000000.dat']

In [14]:
for fname in files:
    for b, dataset in enumerate(['EWOffset','NSOffset','offsetSNR']):
        fname_out = f'{os.path.splitext(fname)[0]}_{dataset}.tif'
        ds = gdal.Open(fname)
        ds = gdal.Translate(fname_out, ds, bandList = [b+1])
        ds = None

# write mintpy file
files = glob.glob('*EWOffset.tif')+glob.glob('*NSOffset.tif')+glob.glob('*offsetSNR.tif')
files.sort()
lines = [f'{f} {f[9:17]} {f[25:33]}\n' for f in files]
with open('metadata.txt', 'w') as f:
    f.writelines(lines)